In [1]:
# Establish connections to the input db and variables spreadsheets

# Import modules
import os, re, math
import arcpy
import pandas as pd
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from dotenv import load_dotenv

# Lot the .env variables
load_dotenv()

# Set the data paths
inputs_path = os.getenv('J111_INPUTS')
outputs_path = os.getenv('J111_OUTPUTS')
spatial_path = os.getenv('J111_SPATIAL')
env_path = os.getenv('J111_ENV')

# Set path to the input variables excel workbook
vars_xlsx = os.path.join(inputs_path, 'Redland_GAM_Input_Vars.xlsx')

# Get full path of Input GDBs
pscap_gdb_path = os.path.join(spatial_path, 'redland_gam_ps_capacity.gdb')
curdev_gdb_path = os.path.join(spatial_path, 'redland_gam_current_dev.gdb')

# Set arcpy workspace
arcpy.env.workspace = os.path.join(env_path, 'J111_redland_gam.gdb')

# Import the property base (with zones applied)
pb_zoned = pd.DataFrame.spatial.from_featureclass(os.path.join(pscap_gdb_path, 'GEN_FC_PropertyBase_ZonesApplied')).drop(['OBJECTID', 'SHAPE'], axis=1)

# Inspect dataframe
pb_zoned.head()

,pbno,pb_part,pb_uid,zone,zone_name,zone_type,property_area,property_zone_area
0,8000000,0.0,8000000-0,MDR,Medium Density Residential,zone,2292.86,2292.86
1,8000001,0.0,8000001-0,MU,Mixed Use,zone,5113.37,5113.37
2,8000002,0.0,8000002-0,TA,Tourist Accommodation,zone,3364.24,3364.24
3,8000003,0.0,8000003-0,MDR,Medium Density Residential,zone,9343.39,9343.39
4,8000004,0.0,8000004-0,CAP-CMR2,Capalaba - Commerical Precinct 2,structure plan,2075.52,2075.52


In [2]:
''' 
----------------------------------
Get zones input table
----------------------------------
'''

# Import zones table from input
zones = pd.read_excel(vars_xlsx, 'PS_Zones').drop(['name', 'source', 'authority'], axis=1)

# Inspect
zones.head()

,code,future_dev,dwelling_propensity,min_frontage,min_area,res_density,dwelling_cap,plot_ratio,conversion_factor
0,CAP-CF,Infill,None,NaN,NaN,0.0,NaN,0.24,0.0
1,CAP-CMR1,Infill,None,NaN,NaN,0.0,NaN,2.50,0.3
2,CAP-CMR2,Infill,None,NaN,NaN,0.0,NaN,2.50,0.3
3,CAP-CMR3,Infill,None,NaN,NaN,0.0,NaN,2.50,0.3
4,CAP-MDR,Infill,Attached,NaN,NaN,124.0,1800.0,0.00,0.3


In [3]:
''' 
----------------------------------
Import tables required for frontage length for analysis
----------------------------------
'''

# Import the required tables from the current development geodatabase
parcel_bounds = pd.DataFrame.spatial.from_featureclass(os.path.join(curdev_gdb_path, 'INPUT_FC_DCDB_SharedBoundaries')).drop(['OBJECTID', 'SHAPE'], axis=1)
pb_all = pd.DataFrame.spatial.from_featureclass(os.path.join(curdev_gdb_path, 'GEN_FC_PropertyBase')).drop(['OBJECTID', 'SHAPE'], axis=1)
pb_agg = pd.DataFrame.spatial.from_table(os.path.join(curdev_gdb_path, 'GEN_TB_DCDB_Aggregates')).drop(['OBJECTID'], axis=1)

# Inspect
parcel_bounds.head()

,segpar,shared_segpar,shared_length,shared_type,shared_name
0,28663092,28663086,18.25,Road,Jonquil Court
1,28663092,28663086,1.75,Road,Jonquil Court
2,28663092,28663093,38.84,Lot,None
3,28663092,28663255,9.44,Lot,None
4,28663092,28663254,10.56,Lot,None


In [4]:
''' 
----------------------------------
Get road frontage lengths for each base property
----------------------------------
'''

# Only select those boundary segments that are a road frontage
frontages = parcel_bounds.query("shared_type == 'Road'").drop(['shared_segpar', 'shared_type', 'shared_name'], axis=1)

# Groupby segpar and get the full length of road frontages for the parcel
frontages = frontages.groupby('segpar').sum().reset_index().rename({'shared_length' : 'frontage'}, axis=1)

# Join frontages to aggregates
agg_frontages = pd.merge(pb_agg, frontages, on='segpar', how='left').dropna()

# Run the frontage calculations on for each aggregate
agg_frontages = agg_frontages.loc[:, ('aggid', 'frontage')].groupby('aggid').sum().reset_index()

# Join the non-aggregate frontages propbase
pb_frontages = pd.merge(pb_all.loc[:, ('pbno', 'segpar', 'aggid')], frontages, on='segpar', how='left').dropna(subset=['frontage'])

# Join the aggregate frontages to the propbase
pb_agg_frontages = pd.merge(pb_all.loc[:, ('pbno', 'segpar', 'aggid')], agg_frontages, on='aggid', how='left').dropna(subset=['frontage'])

# Combine the dataframes
pb_frontages = pd.concat([pb_agg_frontages, pb_frontages]).drop(['segpar', 'aggid'], axis=1)

# Inspect
pb_frontages.head()

,pbno,frontage
0,8000000,64.67
1,8000001,40.49
2,8000002,122.51
3,8000003,235.08
4,8000004,46.25


In [5]:
''' 
----------------------------------
Select properties by zone and threshold

In this step we will exclude any properties (or parts of properties) that are:
- Within an excluded or stable zone
- Do not meet threshold criteria such as minimum frontage or lot size (where stipulated)
----------------------------------
'''

# Drop excluded zones from input table
excluded_ls = ['Excluded', 'Stable']
zones_incl = zones.query("future_dev not in @excluded_ls")

# Join the zone details and filter out the non joins (properties subject to excluded zones)
pb_incl = pd.merge(pb_zoned, zones_incl, how='left', left_on='zone', right_on='code').dropna(subset=['code'])

# Drop redundant columns
pb_incl = pb_incl.drop(['pb_part', 'zone_name', 'zone_type', 'code'], axis=1)

# Split the propbase into those with minimum size criteria and those without
pb_incl_mincrit = pb_incl.query("min_area == min_area") # Select not null via query
pb_incl_open = pb_incl.query("min_area != min_area") # Select null via query

# Drop all properties with a total area under the minimum threshold
pb_incl_mincrit = pb_incl_mincrit.query("property_area > min_area")

# Join the frontage lengths to the table
pb_incl_mincrit = pd.merge(pb_incl_mincrit, pb_frontages, on='pbno', how='left')

# Drop all propertyes with a road frontage under the minimum threshold
pb_incl_mincrit = pb_incl_mincrit.query("frontage > min_frontage")

# Re-combine the filtered tables
pb_incl = pd.concat([pb_incl_mincrit, pb_incl_open]).drop(['frontage', 'min_area', 'min_frontage'], axis=1)

# Inspect
pb_incl.head()

,pbno,pb_uid,zone,property_area,property_zone_area,future_dev,dwelling_propensity,res_density,dwelling_cap,plot_ratio,conversion_factor
0,8000000,8000000-0,MDR,2292.86,2292.86,Infill,Attached,60.0,NaN,0.0,0.3
1,8000001,8000001-0,MU,5113.37,5113.37,Infill,None,0.0,NaN,0.5,0.3
2,8000003,8000003-0,MDR,9343.39,9343.39,Infill,Attached,60.0,NaN,0.0,0.3
4,8000007,8000007-0,MDR,2174.63,2174.63,Infill,Attached,60.0,NaN,0.0,0.3
5,8000010,8000010-0,MDR,1158.12,1158.12,Infill,Attached,60.0,NaN,0.0,0.3


In [6]:
''' 
----------------------------------
Set the future dev type for each property

This step pulls the pre-computed effective developbale area (EDA) for each property
based on the constraint type given in the future_dev column. For infill types an additional
analysis is needed. There are three constraint types for infill: minor, major and subdivision.
To allocate the properties into these categories the unconstrained residential is first calculated.
Any properties with a capacity of over 50 dwellings and a propensity for detached stock are marked
for subdivision. Any properties with a capacity of over 50 dwellings and a propensity for 
attached stock are marked as infill minor. All other residential properties are marked as minor.
Those non-residential properties with a plot_ratio of over 1.0 are major the rest are minor.
----------------------------------
'''

# Split into categories
pb_incl_ind = pb_incl.query("future_dev == 'Industrial'").copy()
pb_incl_inf = pb_incl.query("future_dev == 'Infill'").copy()
pb_incl_nc = pb_incl.query("future_dev == 'New Community'").copy()

# Calculate the unconstrained residential capacity
pb_incl_inf['res_cap'] = pb_incl_inf.apply(lambda row: math.floor((row.property_zone_area / 10000) * row.res_density), axis=1)

# Define a function to set the infill type based on the arguments above
def SetInfillType(row):

    if row.res_cap > 50 and row.dwelling_propensity == 'Detached':
        return 'Subdivision'
    elif row.res_cap > 50 and row.dwelling_propensity == 'Attached':
        return 'Infill Major'
    elif row.res_cap:
        return 'Infill Minor'
    elif row.plot_ratio > 1.0:
        return 'Infill Major'
    else:
        return 'Infill Minor'

# Set the infill type
pb_incl_inf['future_dev'] = pb_incl_inf.apply(lambda row: SetInfillType(row), axis=1)

# Recombine the tables
pb_incl_fdev = pd.concat([pb_incl_inf, pb_incl_nc, pb_incl_ind]).drop(['res_cap'], axis=1)

# Inspect
pb_incl_fdev.head()

,pbno,pb_uid,zone,property_area,property_zone_area,future_dev,dwelling_propensity,res_density,dwelling_cap,plot_ratio,conversion_factor
0,8000000,8000000-0,MDR,2292.86,2292.86,Infill Minor,Attached,60.0,NaN,0.0,0.3
1,8000001,8000001-0,MU,5113.37,5113.37,Infill Minor,None,0.0,NaN,0.5,0.3
2,8000003,8000003-0,MDR,9343.39,9343.39,Infill Major,Attached,60.0,NaN,0.0,0.3
4,8000007,8000007-0,MDR,2174.63,2174.63,Infill Minor,Attached,60.0,NaN,0.0,0.3
5,8000010,8000010-0,MDR,1158.12,1158.12,Infill Minor,Attached,60.0,NaN,0.0,0.3


In [10]:
''' 
----------------------------------
Import effective developable area tables
----------------------------------
'''

# Import the required tables 
eda_ind = pd.DataFrame.spatial.from_table(os.path.join(pscap_gdb_path, 'OUT_TB_AppliedConstraints_Industrial')).drop(['OBJECTID'], axis=1)
eda_infmin = pd.DataFrame.spatial.from_table(os.path.join(pscap_gdb_path, 'OUT_TB_AppliedConstraints_InfillMinor')).drop(['OBJECTID'], axis=1)
eda_infmaj = pd.DataFrame.spatial.from_table(os.path.join(pscap_gdb_path, 'OUT_TB_AppliedConstraints_InfillMajor')).drop(['OBJECTID'], axis=1)
eda_nc = pd.DataFrame.spatial.from_table(os.path.join(pscap_gdb_path, 'OUT_TB_AppliedConstraints_NewCommunity')).drop(['OBJECTID'], axis=1)
eda_subd = pd.DataFrame.spatial.from_table(os.path.join(pscap_gdb_path, 'OUT_TB_AppliedConstraints_Subdivision')).drop(['OBJECTID'], axis=1)

# Add future development type to the tables
eda_ind['future_dev'] = 'Industrial'
eda_infmin['future_dev'] = 'Infill Minor'
eda_infmaj['future_dev'] = 'Infill Major'
eda_nc['future_dev'] = 'New Community'
eda_subd['future_dev'] = 'Subdivision'

# Combine all imported tables
pb_eda = pd.concat([eda_ind, eda_infmaj, eda_infmin, eda_nc, eda_subd]).drop(['property_zone_area', 'constrained_area', 'constrained_area_applied'], axis=1)

# Inspect
pb_eda.head()

,pb_uid,effective_developable_area,future_dev
0,8000002-0,3364.24,Industrial
1,8029015-0,536.95,Industrial
2,8054354-0,1014.61,Industrial
3,8006731-0,718.06,Industrial
4,8032704-0,599.94,Industrial


In [11]:
''' 
----------------------------------
Calculate residential and non-residential development capacity
----------------------------------
'''

# Join the effective developable area calcs onto the pb table via pb_uid and future_dev
pb_incl_eda = pd.merge(pb_incl_fdev, pb_eda, how='left', left_on=['pb_uid', 'future_dev'], right_on=['pb_uid', 'future_dev'])

# Rename eda for convenience
pb_incl_eda =  pb_incl_eda.rename({'effective_developable_area' : 'eda'}, axis=1)

# In the case there is no computed eda (no constraints) then set it to be the property_zone_area
pb_incl_eda = pb_incl_eda.query("eda == eda")
pb_incl_noeda = pb_incl_eda.query("eda != eda")
pb_incl_noeda['eda'] = pb_incl_noeda['property_zone_area']

# Re-combine the tables
pb_incl_cap = pd.concat([pb_incl_noeda, pb_incl_eda])

# Calculate the constrained residential capacity
pb_incl_cap['res_cap'] = pb_incl_cap.apply(lambda row: math.floor((row.eda / 10000) * row.res_density), axis=1)

# Calculate the contrained non-residential capacity
pb_incl_cap['nonres_cap'] = pb_incl_cap.apply(lambda row: row.eda * row.plot_ratio, axis=1)

# Drop extraneous columns
pb_incl_cap = pb_incl_cap.drop(['future_dev', 'res_density', 'plot_ratio'], axis=1)

# Inspect
pb_incl_cap.head()

,pbno,pb_uid,zone,property_area,property_zone_area,dwelling_propensity,dwelling_cap,conversion_factor,eda,res_cap,nonres_cap
0,8000000,8000000-0,MDR,2292.86,2292.86,Attached,NaN,0.3,2292.86,13,0.000
1,8000001,8000001-0,MU,5113.37,5113.37,None,NaN,0.3,5113.37,0,2556.685
2,8000003,8000003-0,MDR,9343.39,9343.39,Attached,NaN,0.3,7010.74,42,0.000
3,8000007,8000007-0,MDR,2174.63,2174.63,Attached,NaN,0.3,2039.38,12,0.000
4,8000010,8000010-0,MDR,1158.12,1158.12,Attached,NaN,0.3,1038.27,6,0.000


In [12]:
''' 
----------------------------------
Reallocate residential capacity for those areas with a dwelling cap and set final capacities
----------------------------------
'''

# Split by those with dwelling cap and otherwise
pb_incl_dcap = pb_incl_cap.query("dwelling_cap == dwelling_cap")
pb_incl_nodcap = pb_incl_cap.query("dwelling_cap != dwelling_cap")

# Get the unique caps by zone
dcap_zones = pb_incl_dcap.loc[:, ('zone', 'dwelling_cap')].drop_duplicates().reset_index(drop=True)

# Get the total latent capacity for each zone
dcap_zones = pd.merge(dcap_zones, pb_incl_dcap.loc[:, ('zone', 'res_cap')].groupby('zone').sum().reset_index(), on='zone', how='left')

# Rename res_cap for visibility
dcap_zones = dcap_zones.rename({'res_cap' : 'latent_cap'}, axis=1)

# Join latent capacity back onto pb table
pb_incl_dcap = pd.merge(pb_incl_dcap, dcap_zones.loc[:, ('zone', 'latent_cap')], on='zone', how='left')

# Get contribution to latent capacity as a fraction
pb_incl_dcap['latent_cap_frac'] =  pb_incl_dcap['res_cap'] / pb_incl_dcap['latent_cap']

# Set the residential capacity to be a fraction of the dwelling cap based on the latent capacity fraction
pb_incl_dcap['res_cap'] = pb_incl_dcap.apply(lambda row: math.floor(row.dwelling_cap * row.latent_cap_frac), axis=1)

# Re-combine the tables
pb_incl_devcap = pd.concat([pb_incl_nodcap, pb_incl_dcap])

# Set the dwellings to attached or detached based on propensity
pb_incl_devcap['det_dwl'] = pb_incl_devcap.apply(lambda row: row.res_cap if row.dwelling_propensity == 'Detached' else 0, axis=1)
pb_incl_devcap['att_dwl'] = pb_incl_devcap.apply(lambda row: row.res_cap if row.dwelling_propensity == 'Attached' else 0, axis=1)

# Drop extraneous columns
pb_incl_devcap = pb_incl_devcap.drop(['property_zone_area', 'dwelling_propensity', 'res_cap', 'dwelling_cap', 'latent_cap', 'latent_cap_frac'], axis=1)

# Rename columns
pb_incl_devcap = pb_incl_devcap.rename({'nonres_cap' : 'nonres_gfa'}, axis=1)

# Inspect
pb_incl_devcap.head()

,pbno,pb_uid,zone,property_area,conversion_factor,eda,nonres_gfa,det_dwl,att_dwl
0,8000000,8000000-0,MDR,2292.86,0.3,2292.86,0.000,0,13
1,8000001,8000001-0,MU,5113.37,0.3,5113.37,2556.685,0,0
2,8000003,8000003-0,MDR,9343.39,0.3,7010.74,0.000,0,42
3,8000007,8000007-0,MDR,2174.63,0.3,2039.38,0.000,0,12
4,8000010,8000010-0,MDR,1158.12,0.3,1038.27,0.000,0,6


In [13]:
''' 
----------------------------------
Allocate non residential capacity to dev groups based on input
----------------------------------
'''
# list non residential columns
nonres_cols = [
    'commercial',
    'community',
    'education',
    'health',
    'industrial',
    'other',
    'retail',
]

# Import non residential zones allocation table
nonres_alloc = pd.read_excel(vars_xlsx, 'PS_Zones_to_DevGroup').drop(['alloc_check'], axis=1).rename({'code' : 'zone'}, axis=1)

# Join non residential columns to devcap
pb_devcap_all = pd.merge(pb_incl_devcap, nonres_alloc, on='zone', how='left')

# Loop through each of the non-res columns and allocate the nonres_gfa
for col in nonres_cols:
    pb_devcap_all[col] = pb_devcap_all['nonres_gfa'] * pb_devcap_all[col]

# Inspect
pb_devcap_all.head()

,pbno,pb_uid,zone,property_area,conversion_factor,eda,nonres_gfa,det_dwl,att_dwl,commercial,community,education,health,industrial,other,retail,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,8000000,8000000-0,MDR,2292.86,0.3,2292.86,0.000,0,13,0.000,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN,NaN
1,8000001,8000001-0,MU,5113.37,0.3,5113.37,2556.685,0,0,511.337,0.0,0.0,0.0,0.0,0.0,2045.348,NaN,NaN,NaN,NaN
2,8000003,8000003-0,MDR,9343.39,0.3,7010.74,0.000,0,42,0.000,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN,NaN
3,8000007,8000007-0,MDR,2174.63,0.3,2039.38,0.000,0,12,0.000,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN,NaN
4,8000010,8000010-0,MDR,1158.12,0.3,1038.27,0.000,0,6,0.000,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN,NaN


In [14]:
''' 
----------------------------------
Aggregate sub property capacities to propbase 
----------------------------------
'''

# Get list of unique zones for each property
pb_devcap_ls = pb_devcap_all.loc[:, ('pbno', 'zone')].groupby('pbno').agg(set).reset_index()

# Convert the zones to a string
pb_devcap_ls['zone'] = pb_devcap_ls['zone'].str.join(',')

# Get property area and conversion factor for each property
pb_devcap_max = pb_devcap_all.loc[:, ('pbno', 'property_area', 'conversion_factor')].groupby('pbno').agg('max').reset_index().drop(['pbno'], axis=1)

# Get summary of development capacity calcs for each property
pb_devcap_sum = pb_devcap_all.loc[:, ('pbno', 'eda', 'det_dwl', 'att_dwl', 'nonres_gfa')].groupby('pbno').agg('sum').reset_index().drop(['pbno'], axis=1)

# Get summary of non-residential calcs for each property
pb_devcap_sumnonres = pb_devcap_all.loc[:, ('pbno',) + tuple(nonres_cols)].groupby('pbno').agg('sum').reset_index().drop(['pbno'], axis=1)

# Combine the above tables horizontally
pb_devcap = pd.concat([pb_devcap_ls, pb_devcap_max, pb_devcap_sum, pb_devcap_sumnonres], axis=1)

# -------------------------------------
# EXCEPTION!!!

# Force all character residential dwellings to a single detached dwelling per property
pb_devcap['det_dwl'] = pb_devcap.apply(lambda row: 1.0 if row.zone == 'CR' and row.det_dwl > 1 else row.det_dwl, axis=1)

# -------------------------------------

# Set total dwellings column
pb_devcap['tot_dwl'] = pb_devcap['det_dwl'] + pb_devcap['att_dwl']

# Inspect
pb_devcap.head()

,pbno,zone,property_area,conversion_factor,eda,det_dwl,att_dwl,nonres_gfa,commercial,community,education,health,industrial,other,retail,tot_dwl
0,8000000,MDR,2292.86,0.3,2292.86,0.0,13,0.000,0.000,0.0,0.0,0.0,0.0,0.000,0.000,13.0
1,8000001,MU,5113.37,0.3,5113.37,0.0,0,2556.685,511.337,0.0,0.0,0.0,0.0,0.000,2045.348,0.0
2,8000002,TA,3364.24,0.3,3364.24,0.0,0,672.848,0.000,0.0,0.0,0.0,0.0,672.848,0.000,0.0
3,8000003,MDR,9343.39,0.3,7010.74,0.0,42,0.000,0.000,0.0,0.0,0.0,0.0,0.000,0.000,42.0
4,8000004,CAP-CMR2,2075.52,0.3,2075.52,0.0,0,5188.800,5188.800,0.0,0.0,0.0,0.0,0.000,0.000,0.0


In [15]:
''' 
----------------------------------
Import and prepare current development stats
----------------------------------
'''

# Import current development metrics from geodatabase
pb_curdev_lyr = pd.DataFrame.spatial.from_featureclass(os.path.join(curdev_gdb_path, 'OUT_FC_PropertyBase_CurrentDev')).drop(['OBJECTID'], axis=1)

# Add the long term accommodation calcs to attached dwellings and drop column
pb_curdev_lyr['att_dwl'] = pb_curdev_lyr['att_dwl'] + pb_curdev_lyr['accom_long']
pb_curdev_lyr = pb_curdev_lyr.drop(['accom_long'], axis=1)

# Summarise current non-residential development into broader development groups

pb_curdev_lyr['commercial'] = pb_curdev_lyr['office']
pb_curdev_lyr['community'] = pb_curdev_lyr['emer_serv'] + pb_curdev_lyr['pl_of_ass']
pb_curdev_lyr['education'] = pb_curdev_lyr['childcare'] + pb_curdev_lyr['education']
pb_curdev_lyr['health'] = pb_curdev_lyr['care_accom'] + pb_curdev_lyr['hospital'] + pb_curdev_lyr['health_serv']
pb_curdev_lyr['industrial'] = pb_curdev_lyr['heavy_ind'] + pb_curdev_lyr['light_ind']
pb_curdev_lyr['other'] = pb_curdev_lyr['accom_short'] + pb_curdev_lyr['other'] + pb_curdev_lyr['rural'] + pb_curdev_lyr['indoor_ent'] + pb_curdev_lyr['indoor_rec']
pb_curdev_lyr['retail'] = pb_curdev_lyr['bulk_goods'] + pb_curdev_lyr['food_drink'] + pb_curdev_lyr['retail']

# Drop redundant non-residential columns
drop_nonres_col = [
    'office',
    'accom_short',
    'emer_serv',
    'pl_of_ass',
    'childcare',
    'care_accom',
    'hospital',
    'health_serv',
    'heavy_ind',
    'light_ind',
    'rural',
    'bulk_goods',
    'food_drink',
    'indoor_ent',
    'indoor_rec'
]
pb_curdev_lyr = pb_curdev_lyr.drop(drop_nonres_col, axis=1)

# Copy current development layer
pb_curdev = pb_curdev_lyr.copy()

# Get total non-residential gfa by summing nonres columns
pb_curdev['nonres_gfa'] = pb_curdev.loc[:, tuple(nonres_cols)].sum(axis=1)

# Set total dwellings column
pb_curdev['tot_dwl'] = pb_curdev['det_dwl'] + pb_curdev['att_dwl']

# Inspect
pb_curdev.head()

,pbno,sa2_name,prop_use,det_dwl,att_dwl,education,other,retail,SHAPE,commercial,community,health,industrial,nonres_gfa,tot_dwl
0,8000000,Cleveland,"INF021,RES001",0.0,9.0,0.0,0.0,0.0,"{""rings"": [[[528400.0832000002, 6956055.8798],...",0.00,0.0,0.0,0.0,0.00,9.0
1,8000001,Cleveland,"INF020,IND004",0.0,0.0,0.0,0.0,0.0,"{""rings"": [[[525679.0065000001, 6955447.2788],...",3404.31,0.0,0.0,0.0,3404.31,0.0
2,8000002,Redland Islands,"INF021,RES001",0.0,18.0,0.0,0.0,0.0,"{""rings"": [[[553508.1963, 6965299.8267], [5535...",0.00,0.0,0.0,0.0,0.00,18.0
3,8000003,Cleveland,"INF021,RES001",0.0,146.0,0.0,0.0,0.0,"{""rings"": [[[526394.1940000001, 6955803.3852],...",0.00,0.0,0.0,0.0,0.00,146.0
4,8000004,Capalaba,"INF020,IND004",0.0,0.0,0.0,0.0,0.0,"{""rings"": [[[518662.1150000002, 6955518.7645],...",803.46,0.0,0.0,0.0,803.46,0.0


In [16]:
''' 
----------------------------------
Get list of residential and non residential conversions
----------------------------------
'''

# ----------------------------------
# Residential conversions

# Map the dwelling capacity (tot_dwl) to the current count of dwellings (cur_dwl)
pb_devcap_resconv = pd.merge(
    pb_devcap.loc[:, ('pbno', 'tot_dwl', 'conversion_factor')], 
    pb_curdev.loc[:, ('pbno', 'tot_dwl')].rename({'tot_dwl' : 'cur_dwl'}, axis=1),
on='pbno', how='left').fillna(0)

# If the dwelling capacity exceeds the conversion factor (cur_dwl + (cur_dwl * conversion_factor)) and the are less than 10 existing dwellings mark as conversion
pb_devcap_resconv['res_conv'] = pb_devcap_resconv.apply(lambda row: True if row.tot_dwl > row.cur_dwl + (row.cur_dwl * row.conversion_factor) and row.cur_dwl < 10 else False,axis=1)

# Get a list of all the properties that have a residential conversion
pb_resconv_ls = pb_devcap_resconv.query("res_conv == True")['pbno'].to_list()

# ----------------------------------
# Non residential conversions

# Map the gfa capacity (nonres_gfa) to the current gfa (cur_nonres_gfa)
pb_devcap_nonresconv = pd.merge(
    pb_devcap.loc[:, ('pbno', 'nonres_gfa', 'conversion_factor')], 
    pb_curdev.loc[:, ('pbno', 'nonres_gfa')].rename({'nonres_gfa' : 'cur_nonres_gfa'}, axis=1),
on='pbno', how='left').fillna(0)

# If the non-residential gfa exceeds the conversion factor mark as conversion
pb_devcap_nonresconv['nonres_conv'] = pb_devcap_nonresconv.apply(lambda row: True if row.nonres_gfa > row.cur_nonres_gfa + (row.cur_nonres_gfa * row.conversion_factor) else False,axis=1)

# Get a list of all the properties that have a non-residential conversion
pb_nonresconv_ls = pb_devcap_nonresconv.query("nonres_conv == True")['pbno'].to_list()

# ----------------------------------

# Inspect
'{} Residential conversions | {} Non-residential conversions'.format(len(pb_resconv_ls), len(pb_nonresconv_ls))

'3796 Residential conversions | 970 Non-residential conversions'

In [17]:
''' 
----------------------------------
Prepare final planned capacity layer (handling conversions)
----------------------------------
'''

# Compile the residential capacities 
pb_cap_res = pb_devcap.query("pbno in @pb_resconv_ls").loc[:, ('pbno', 'det_dwl', 'att_dwl')]

# Compile the non-residential capacities 
pb_cap_nonres =  pb_devcap.query("pbno in @pb_nonresconv_ls").loc[:, ('pbno',) + tuple(nonres_cols)]

# Get list of unique zones for each property
pb_cap_dtls = pb_zoned.loc[:, ('pbno', 'zone')].groupby('pbno').agg(set).reset_index()

# Convert the zones to a string
pb_cap_dtls['zone'] = pb_cap_dtls['zone'].str.join(',')

# Join the residential and non-residential capacities together
pb_cap = pb_curdev_lyr.drop(['prop_use'], axis=1).set_index('pbno')

# Update the residential capacities
pb_cap.update(pb_cap_res.set_index('pbno'))

# Update the non-residential capacities
pb_cap.update(pb_cap_nonres.set_index('pbno'))

# Create a new zones column with empty values
pb_cap['zone'] = None

# Update the zone details
pb_cap.update(pb_cap_dtls.set_index('pbno'))

# Set conversion states
pb_cap['res_conv'] = pb_cap.apply(lambda row: True if row.name in pb_resconv_ls else False, axis=1)
pb_cap['nonres_conv'] = pb_cap.apply(lambda row: True if row.name in pb_nonresconv_ls else False, axis=1)

# Create new area columns
pb_cap['property_area'] = 0.0
pb_cap['eda'] = 0.0

# Update the areas
pb_cap.update(pb_devcap.loc[:, ('pbno', 'property_area', 'eda')].set_index('pbno'))

# Reset the index
pb_cap = pb_cap.reset_index()

# Get total non-residential gfa by summing nonres columns
pb_cap['nonres_gfa'] = pb_cap.loc[:, tuple(nonres_cols)].sum(axis=1)

# Set the order for columns
col_order = ['pbno', 'sa2_name', 'zone', 'property_area', 'eda', 'res_conv', 'nonres_conv', 'det_dwl', 'att_dwl', 'nonres_gfa'] + nonres_cols + ['SHAPE']
pb_cap = pb_cap[col_order]

# Inspect
pb_cap.head()

,pbno,sa2_name,zone,property_area,eda,res_conv,nonres_conv,det_dwl,att_dwl,nonres_gfa,commercial,community,education,health,industrial,other,retail,SHAPE
0,8000000,Cleveland,MDR,2292.86,2292.86,True,False,0.0,13.0,0.000,0.00,0.0,0.0,0.0,0.0,0.000,0.0,"{""rings"": [[[528400.0832000002, 6956055.8798],..."
1,8000001,Cleveland,MU,5113.37,5113.37,False,False,0.0,0.0,3404.310,3404.31,0.0,0.0,0.0,0.0,0.000,0.0,"{""rings"": [[[525679.0065000001, 6955447.2788],..."
2,8000002,Redland Islands,TA,3364.24,3364.24,False,True,0.0,18.0,672.848,0.00,0.0,0.0,0.0,0.0,672.848,0.0,"{""rings"": [[[553508.1963, 6965299.8267], [5535..."
3,8000003,Cleveland,MDR,9343.39,7010.74,False,False,0.0,146.0,0.000,0.00,0.0,0.0,0.0,0.0,0.000,0.0,"{""rings"": [[[526394.1940000001, 6955803.3852],..."
4,8000004,Capalaba,CAP-CMR2,2075.52,2075.52,False,True,0.0,0.0,5188.800,5188.80,0.0,0.0,0.0,0.0,0.000,0.0,"{""rings"": [[[518662.1150000002, 6955518.7645],..."


In [18]:
''' 
----------------------------------
Get the development capacity summary by SA2
----------------------------------
'''
# Group development statistics by SA2
sa2_cap = pb_cap.loc[:, ('sa2_name', 'det_dwl', 'att_dwl',) + tuple(nonres_cols)].groupby('sa2_name').agg('sum')

# Inspect
sa2_cap.head(20)

,det_dwl,att_dwl,commercial,community,education,health,industrial,other,retail
sa2_name,,,,,,,,,
Alexandra Hills,5778.0,1249.0,1925.7284,3230.3554,114479.126,6056.442,0.000,23304.687,34326.1606
Birkdale,5262.0,1790.0,11239.3384,6600.6188,34429.603,17375.589,17.563,10029.768,51012.2276
Capalaba,5539.0,5704.0,312858.8704,25760.8776,93857.331,6959.001,331645.397,13965.021,637834.0606
Cleveland,4284.0,9268.0,82433.7046,33924.3782,54339.794,228108.398,143303.801,24135.188,205877.6544
Ormiston,2049.0,991.0,43698.2700,9847.1590,74844.578,0.000,32451.078,2313.780,55545.3710
Redland Bay,11586.0,1607.0,30369.2609,24890.1190,16337.797,3546.627,161945.839,83379.800,132859.6246
Redland Islands,8450.0,1129.0,34714.3666,34464.8246,53674.753,2765.539,153171.888,42120.126,132912.3944
Sheldon - Mount Cotton,2714.0,64.0,3332.1744,2360.8894,47064.468,0.000,39349.317,89492.032,14408.4406
Thorneside,1016.0,931.0,1860.8300,690.3330,0.000,0.000,32801.064,11290.055,8213.1800


In [19]:
''' 
----------------------------------
Output Summary
----------------------------------
'''
# Output SA2 metrics to csv
sa2_cap.to_csv(os.path.join(outputs_path, 'OUT_Redland_SA2_DevCapacity.csv'))

In [20]:
''' 
----------------------------------
Output Features
----------------------------------
'''
# Output development capacity back into geodatabase
pb_cap.spatial.to_featureclass(os.path.join(pscap_gdb_path, 'OUT_FC_PropertyBase_DevCapacity'), overwrite=True)

'G:\\Shared drives\\PIESolutions_03_Projects\\J000111 - Redlands planning assumption update\\06_Working Documents\\00_GIS Directory\\00_Data\\gam_model_run\\redland_gam_ps_capacity.gdb\\OUT_FC_PropertyBase_DevCapacity'